In [27]:
from sage.all import *

D = RealField(127)

x = '10'
y = '18'

x0 = D(x)
y0 = D(y)
x0p1 = x0 + D(1)
x0p1ln = x0p1.log()

x0p1ln_y0 = x0p1ln * y0

print(f"LOG {x0p1ln_y0}")

z = x0p1ln_y0.exp() - D(1)

print(z)


LOG 43.16211491037066979311498440337232740
5.559917313492231480000000000000000002e18


In [28]:
import struct

# Example array of big floats in SageMath
bigfloats = [1,
0.5,
0.166666666666666657414808128123695496469736099243164,
4.1666666666666664353702032030923874117434024810791e-2,
8.333333941167623062917790832671016687527298927307e-3,
1.38888903862928589014180680294430203502997756004333e-3]

def double_to_hex(f):
    # Convert float to IEEE 754 double hex representation
    packed = struct.pack('>d', float(f))  # big-endian double
    return '0x' + packed.hex()

hex_array = [double_to_hex(RR(x)) for x in bigfloats]

print(", ".join(hex_array))


0x3ff0000000000000, 0x3fe0000000000000, 0x3fc5555555555555, 0x3fa5555555555555, 0x3f81111125f3a235, 0x3f56c16c3fea7746


In [29]:
import struct
from sage.all import *

R = RealField(52)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = R(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

DD = RealField(120)

# Input RR array (high precision)
bigfloats = [
'1',
'1.00000000000000000000000000010659482981798922020626',
'0.50000000000000000000000000003235562306570556233134',
'0.16666666666666666661650065088154907094781865583483',
'4.1666666666666666656813389760732706689332161864754e-2',
'8.333333339235663398545366857448291597622040747153e-3',
'1.38888888977721761402414540629642690134205558400704e-3',
]

# Split first 4 as DD pairs
dd_pairs = [split_double_double(DD(x)) for x in bigfloats]

# Convert to hex
hex_pairs = [(double_to_hex(hi), double_to_hex(lo)) for hi, lo in dd_pairs]

# Print as flat list
flat_hex = [x for pair in hex_pairs for x in pair]
print(", ".join(flat_hex))

d_pi = split_double_double(DD.pi())
pi_hi = double_to_hex(d_pi[1])
pi_lo = double_to_hex(d_pi[0])
print(pi_hi)
print(pi_lo)

0x0000000000000000, 0x3ff0000000000000, 0x3a20e40000000000, 0x3ff0000000000000, 0x3a04820000000000, 0x3fe0000000000000, 0xbc756423c5338a66, 0x3fc5555555555556, 0xbc5560f74db5556c, 0x3fa5555555555556, 0x3c3648eca89bc6ac, 0x3f8111111144fbee, 0xbbd53d924ae90c8c, 0x3f56c16c16ffeecc
0x400921fb54442d18
0x3ca1a62633145c07


In [47]:
import struct
from sage.all import *

terms = 10

var('x')
f = log(1 + x)
t = f.series(x == 0, terms)  # up to x^9 (9 terms)
print(t)

R = RealField(52)

DD = RealField(120)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = R(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def vetkamp_split(a):
    """
    Split a double-precision float into two parts using the Dekker/Vetkamp trick.
    Returns (hi, lo) such that a = hi + lo and hi contains the leading bits.
    """
    # Use 2^27 + 1 as the constant for f64 splitting (Dekker's method)
    C = float(2**27 + 1)
    t1 = C * a
    t2 = a - t1
    r_hi = t1 + t2
    r_lo = a - r_hi
    return r_lo, r_hi

print("\nCoefficients:")
for i in range(1, terms):  # log(1+x) has no constant term
    coeff = t.coefficient(x**i)
    k = split_double_double(DD(coeff))
    hi = double_to_hex(k[1])
    lo = double_to_hex(k[0])
    print(f"({lo}, {hi}),")
    # print(f"x^{i}: {coeff}")

print("\nCoefficients double:")
for i in range(1, terms): 
    coeff = t.coefficient(x**i)
    k = split_double_double(DD(coeff))
    hi = double_to_hex(k[1])
    print(f"({hi}),")
    # print(f"x^{i}: {coeff}")

1*x + (-1/2)*x^2 + 1/3*x^3 + (-1/4)*x^4 + 1/5*x^5 + (-1/6)*x^6 + 1/7*x^7 + (-1/8)*x^8 + 1/9*x^9 + Order(x^10)

Coefficients:
(0x0000000000000000, 0x3ff0000000000000),
(0x0000000000000000, 0xbfe0000000000000),
(0xbe25555558000000, 0x3fd5555558000000),
(0x0000000000000000, 0xbfd0000000000000),
(0x3e099999a0000000, 0x3fc9999998000000),
(0x3e15555558000000, 0xbfc5555558000000),
(0x3e12492490000000, 0x3fc2492490000000),
(0x0000000000000000, 0xbfc0000000000000),
(0xbe0c71c720000000, 0x3fbc71c720000000),

Coefficients double:
(0x3ff0000000000000),
(0xbfe0000000000000),
(0x3fd5555555555556),
(0xbfd0000000000000),
(0x3fc999999999999a),
(0xbfc5555555555556),
(0x3fc2492492492492),
(0xbfc0000000000000),
(0x3fbc71c71c71c71c),


In [55]:
bigfloats = [
'1',
'-0.5',
'0.333333333333333333333333333333332306170696326807545036811763954705430113012454285126295872032642364501953125',
'-0.249999999999999999999999999999682606745164983531416374835062003977904920848374104025424458086490631103515625',
'0.200000000000000000000000002387949136540462961767856959621631279485666488060502388179884292185306549072265625',
'-0.166666666666666666666667393986043856317874326241244099980138147656764935344853029164369218051433563232421875',
'0.1428571428571428571408163599106053233656149391100956916031132384707624805031400683219544589519500732421875',
'-0.12499999999999999937832028113316696072215483698725704640401028516598358475420127433608286082744598388671875',
'0.11111111111111193038972057718902279427957977404456151551405321587537126937217379918365622870624065399169921875',
'-0.1000000000002509880241711464546716390478971261945224395226512311622528439425394708450767211616039276123046875',
'9.0909090751649750743304935100388324903899779563569833378440453413964206230701847744057886302471160888671875e-2',
'-8.3333285000235375895682579496227015767268553492735970464138327284953078333273879252374172210693359375e-2',
'7.69350598662831325463995536139570999801209091083709001164372138083062668822975638249772600829601287841796875e-2',
'-7.5007421463363447931295368457235624520969053746502721052817207365859264456275923294015228748321533203125e-2',
]

# Split first 4 as DD pairs
dd_pairs = [split_double_double(DD(x)) for x in bigfloats]

# Convert to hex
hex_pairs = [(double_to_hex(hi), double_to_hex(lo)) for hi, lo in dd_pairs]

# Print as flat list
flat_hex = [x for pair in hex_pairs for x in pair]
print(", ".join(flat_hex))

0x0000000000000000, 0x3ff0000000000000, 0x0000000000000000, 0xbfe0000000000000, 0xbc85555555555556, 0x3fd5555555555556, 0x3999c00000000000, 0xbfd0000000000000, 0xbc6999999981f36e, 0x3fc999999999999a, 0x3c7555554743d27d, 0xbfc5555555555556, 0x3c6247f02d51c658, 0x3fc2492492492492, 0x3c26ef9953cc6840, 0xbfc0000000000000, 0xbc60a66163a63ed2, 0x3fbc71c71c71c758, 0x3c69d1ae7ae4abd1, 0xbfb999999999e040, 0x3c67d7eee3debe26, 0x3fb745d173affb86, 0xbc3ef276254f5c00, 0xbfb5555485be87b6, 0x3c51f94d5ec7053e, 0x3fb3b2041e0a9ef8, 0x3c3d1e0180b9b780, 0xbfb333afb6247696


In [57]:
R = RealField(106)  # ~106 bits = double-double
coeffs = [R(s) for s in bigfloats]

# Polynomial evaluation function (Horner's method)
def dd_polyval(x, coeffs):
    acc = R(0)
    for c in reversed(coeffs):
        acc = acc * x + c
    return acc * x

# Interval and test point
x = R('0.00196075439453125')  # small x near 0 for log(1 + x)
true_val = log(1 + x)
approx = dd_polyval(x, coeffs)

# Compute error
abs_err = abs(approx - true_val)
rel_err = abs_err / abs(true_val)

# Convert to ULPs of 53-bit double precision
ulp = 2^(-53)
abs_ulp = abs_err / ulp
rel_ulp = rel_err / ulp

# Display
print("x =", x)
print("Approximation:", approx)
print("True log(1 + x):", true_val)
print("Absolute error (ULPs):", abs_ulp)
print("Relative error (ULPs):", rel_ulp)


x = 0.001960754394531250000000000000000
Approximation: 0.001958834624688574780016617412059
True log(1 + x): 0.001958834624688574780016617412059
Absolute error (ULPs): -0.0000000000000000000000000000000
Relative error (ULPs): -0.0000000000000000000000000000000
